In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

from obspy.core import UTCDateTime as UT

The catalogs were retrieved from the NEIC PDE database, available at ftp://hazards.cr.usgs.gov/NEICPDE/quakeml/.
These catalogs were turned into csv files that contain the phase for
information for a given earthquake at a given station.

In [63]:
df = pd.read_csv('quakeml/201330_cat_quakeml.csv')
df = df.fillna('NaN')                       # small detail
df = df[(df['Pn']!='NaN')&(df['Sn']!='NaN')]
df

,Unnamed: 0,network,station,channel,P,Pg,Pn,S,Sg,Sn,...,origin_latitude,origin_latitude_uncertainty,origin_longitude,origin_longitude_uncertainty,depth,depth_uncertainty,origin_ID,distance,azimuth,takeoff_angle
12,12,AK,BRLK,BH,NaN,NaN,2013-07-23T01:13:34.120000Z,NaN,NaN,2013-07-23T01:14:01.100000Z,...,60.7970,0.0333,-146.8330,0.0162,16700.0,5900.0,20130723011259_60_-146,2.28468,244.4410,46.0
24,24,AK,SCRK,BH,NaN,NaN,2013-07-23T01:13:51.520000Z,NaN,NaN,2013-07-23T01:14:31.220000Z,...,60.7970,0.0333,-146.8330,0.0162,16700.0,5900.0,20130723011259_60_-146,3.45514,21.2993,50.3
25,25,AK,PIN,BH,NaN,NaN,2013-07-23T01:13:49.440000Z,NaN,NaN,2013-07-23T01:14:28.980000Z,...,60.7970,0.0333,-146.8330,0.0162,16700.0,5900.0,20130723011259_60_-146,3.33572,99.2936,50.3
27,27,AK,RC01,BH,NaN,NaN,2013-07-23T01:13:22.860000Z,NaN,NaN,2013-07-23T01:13:41.740000Z,...,60.7970,0.0333,-146.8330,0.0162,16700.0,5900.0,20130723011259_60_-146,1.44846,282.9390,46.0
146,146,NZ,BKZ,HH,NaN,NaN,2013-07-23T02:03:55.370000Z,NaN,NaN,2013-07-23T02:04:30.960000Z,...,-41.6449,0.0360,174.2226,0.0269,7990.0,6000.0,wel_locationdisp_20130818230334_-42_174,3.02111,35.7473,50.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10450,10450,IR,WSAR,SH,NaN,NaN,2013-07-29T13:09:37.880000Z,NaN,NaN,2013-07-29T13:11:31.870000Z,...,23.2863,0.0495,70.4026,0.0454,20000.0,6200.0,b000iztl,10.82330,272.0830,58.4
10472,10472,IR,WSAR,SH,NaN,NaN,2013-07-29T14:54:29.750000Z,NaN,NaN,2013-07-29T14:55:08.750000Z,...,24.6677,0.1133,62.3725,0.1030,25560.0,4900.0,c000irpf,3.70678,248.2060,59.0
10515,10515,AK,TGL,BH,NaN,NaN,2013-07-29T15:38:05.400000Z,NaN,NaN,2013-07-29T15:38:54.340000Z,...,61.5800,0.0405,-152.0530,0.0193,111200.0,4500.0,20130729153658_61_-152,4.54316,96.4394,83.2
10527,10527,JP,CBIJ,BH,NaN,NaN,2013-07-29T16:08:01.420000Z,NaN,NaN,2013-07-29T16:09:38.970000Z,...,36.4513,0.0791,140.7665,0.0637,59270.0,9400.0,b000iztv,9.40510,172.2390,85.4


In [60]:
df.columns

Index(['Unnamed: 0', 'network', 'station', 'channel', 'P', 'Pg', 'Pn', 'S',
       'Sg', 'Sn', 'P_weight', 'Pg_weight', 'Pn_weight', 'S_weight',
       'Sg_weight', 'Sn_weight', 'P_evaluation_mode', 'Pg_evaluation_mode',
       'Pn_evaluation_mode', 'S_evaluation_mode', 'Sg_evaluation_mode',
       'Sn_evaluation_mode', 'origin_time', 'origin_time_uncertainty',
       'origin_latitude', 'origin_latitude_uncertainty', 'origin_longitude',
       'origin_longitude_uncertainty', 'depth', 'depth_uncertainty',
       'origin_ID', 'distance', 'azimuth', 'takeoff_angle'],
      dtype='object')

Later, the download of the waveforms for each of those traces is attempted via Obspy, by querying all of the servers and networks available.
The names of the waveforms is like follows.


In [64]:
names=df['network']+'.'+df['station']+'.'+df['Pn']+'.Pn.mseed'

In [72]:
for name in names[:10]:
    print('mseedfiles/'+name)
    try:
        st = read('mseedfiles/'+name)
        print(st)
    except Exception as e:
        print('waveform does not exist')

mseedfiles/AK.BRLK.2013-07-23T01:13:34.120000Z.Pn.mseed
waveform does not exist
mseedfiles/AK.SCRK.2013-07-23T01:13:51.520000Z.Pn.mseed
waveform does not exist
mseedfiles/AK.PIN.2013-07-23T01:13:49.440000Z.Pn.mseed
waveform does not exist
mseedfiles/AK.RC01.2013-07-23T01:13:22.860000Z.Pn.mseed
waveform does not exist
mseedfiles/NZ.BKZ.2013-07-23T02:03:55.370000Z.Pn.mseed
waveform does not exist
mseedfiles/NZ.URZ.2013-07-23T02:04:07.360000Z.Pn.mseed
waveform does not exist
mseedfiles/NZ.BFZ.2013-07-23T02:03:36.920000Z.Pn.mseed
waveform does not exist
mseedfiles/NZ.HIZ.2013-07-23T02:03:58.780000Z.Pn.mseed
waveform does not exist
mseedfiles/NZ.LBZ.2013-07-23T02:04:08.380000Z.Pn.mseed
waveform does not exist
mseedfiles/NZ.KHZ.2013-07-23T02:03:26.170000Z.Pn.mseed
waveform does not exist


In [70]:
wavenames=glob.glob('mseedfiles/*2013-07-*Pn.mseed')

In [73]:
sorted(wavenames[:10])

['mseedfiles/G.RER.2013-07-27T23:00:05.980000Z.Pn.mseed',
 'mseedfiles/GE.UGM.2013-07-25T08:58:23.530000Z.Pn.mseed',
 'mseedfiles/GS.MPK01.2013-07-27T19:14:20.960000Z.Pn.mseed',
 'mseedfiles/GT.CPUP.2013-07-23T17:30:28.900000Z.Pn.mseed',
 'mseedfiles/IN.PBA.2013-07-29T22:23:47.820000Z.Pn.mseed',
 'mseedfiles/JP.JHJ2.2013-07-29T16:06:42.390000Z.Pn.mseed',
 'mseedfiles/MN.TIR.2013-07-24T13:01:27.330000Z.Pn.mseed',
 'mseedfiles/TA.061Z.2013-07-31T23:22:37.950000Z.Pn.mseed',
 'mseedfiles/US.OGNE.2013-07-27T23:19:53.740000Z.Pn.mseed',
 'mseedfiles/US.WVOR.2013-07-23T18:42:23.710000Z.Pn.mseed']